In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline

In [3]:
df = pd.read_csv('cardekho_dataset.csv', index_col=[0])

In [4]:
df.head()

,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


#### Data Cleaning

In [5]:
## Check Null Values
df.isnull().sum()

car_name             0
brand                0
model                0
vehicle_age          0
km_driven            0
seller_type          0
fuel_type            0
transmission_type    0
mileage              0
engine               0
max_power            0
seats                0
selling_price        0
dtype: int64

In [6]:
## Remove Unnecesary Columns
df.drop('car_name', axis=1, inplace=True)
df.drop('brand', axis=1, inplace=True)

In [7]:
df.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [8]:
df['model'].unique()

array(['Alto', 'Grand', 'i20', 'Ecosport', 'Wagon R', 'i10', 'Venue',
       'Swift', 'Verna', 'Duster', 'Cooper', 'Ciaz', 'C-Class', 'Innova',
       'Baleno', 'Swift Dzire', 'Vento', 'Creta', 'City', 'Bolero',
       'Fortuner', 'KWID', 'Amaze', 'Santro', 'XUV500', 'KUV100', 'Ignis',
       'RediGO', 'Scorpio', 'Marazzo', 'Aspire', 'Figo', 'Vitara',
       'Tiago', 'Polo', 'Seltos', 'Celerio', 'GO', '5', 'CR-V',
       'Endeavour', 'KUV', 'Jazz', '3', 'A4', 'Tigor', 'Ertiga', 'Safari',
       'Thar', 'Hexa', 'Rover', 'Eeco', 'A6', 'E-Class', 'Q7', 'Z4', '6',
       'XF', 'X5', 'Hector', 'Civic', 'D-Max', 'Cayenne', 'X1', 'Rapid',
       'Freestyle', 'Superb', 'Nexon', 'XUV300', 'Dzire VXI', 'S90',
       'WR-V', 'XL6', 'Triber', 'ES', 'Wrangler', 'Camry', 'Elantra',
       'Yaris', 'GL-Class', '7', 'S-Presso', 'Dzire LXI', 'Aura', 'XC',
       'Ghibli', 'Continental', 'CR', 'Kicks', 'S-Class', 'Tucson',
       'Harrier', 'X3', 'Octavia', 'Compass', 'CLS', 'redi-GO', 'Glanza',
       

In [10]:
## Getting All Different Types OF Features

num_features = [feature for feature in df.columns if df[feature].dtype !='O']
print("Num of Numerical Feature:", len(num_features))

cat_features = [feature for feature in df.columns if df[feature].dtype == 'O']
print("Num of Categorical Feature :", len(cat_features))

discrete_features =[feature for feature in num_features if len(df[feature].unique())<=25]
print("Num of Descrete Feature :", len(discrete_features))

continues_features =[feature for feature in num_features if feature not in discrete_features]
print("Num Of Continues Feature :", len(continues_features))

Num of Numerical Feature: 7
Num of Categorical Feature : 4
Num of Descrete Feature : 2
Num Of Continues Feature : 5


In [11]:
## Independent and dependent features

from sklearn.model_selection import train_test_split
X =df.drop(['selling_price'], axis=1)
y = df['selling_price']

In [12]:
X.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5


#### Feature Encoding ANd Scalling

In [13]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X['model']=le.fit_transform(X['model'])

In [14]:
## Create column transformer with 3 types of transformer

num_features =X.select_dtypes(exclude="object").columns
onehot_columns = ['selling_type', 'fuel_type', 'transmission_type']


from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder(drop='first')

preprocessor = ColumnTransformer(
    [
        ('OneHotEncoder', oh_transformer, onehot_columns),
        ("StandardScaler", numeric_transformer, num_features)

    ],remainder='passthrough'
)

In [15]:
## Seprate parameter into train test
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [16]:
X_train.shape, X_test.shape

((12328, 10), (3083, 10))

#### Model Training Model Selection

In [17]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [18]:
## Create a Function to Evaluate the Model

def evaluate_model(true, predicted):
    mae = mean_squared_error(true,predicted)
    mse = mean_squared_error(true,predicted)
    rmse = np.sqrt(mean_squared_error(true,predicted))
    r2_square = r2_square(true,predicted)
    return mae, rmse, r2_square

In [ ]:
## Beginning Model Training

models = {
    "Linear Regression":LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor":RandomForestRegressor(),

}

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train,y_train)

    y_train_pred =model.predict(X_train)
    y_test_pred =model.predict(X_test)

    ## Evaluating the model train and test datasets
    model_train_mae, model_train_rmse, model_train_r2 = evaluate_model(y_train,y_train_pred)
    model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(y_test,y_test_pred)

    print(list(models.keys())[i])

    print("Model Performance For Traning Set")
    print("Mean Absolute Error :", model_train_mae)
    print("Root Mean Squared Error :", model_train_rmse)
    print("R2 Score :", model_train_r2)
    

    print("----------------------------------")

    print("Model Performance For Traning Set")
    print("Mean Absolute Error :", model_test_mae)
    print("Root Mean Squared Error :", model_test_rmse)
    print("R2 Score :", model_test_r2)
    
    print('='*35)
    print('\n')

ValueError: could not convert string to float: 'Dealer'

In [21]:
knn_params = {"n_neighbours": [2,3,10,20,40 ,50]}
rf_params = {
    "max_depth": [5,8,15,None, 8],
    "max_features": [5,7,"auto", 8],
    "min_samples_split": [2,8,15,20],
    "n_estimators": [100,200,500,1000]
}

In [22]:
## MOdels LIst for Hyperparamete Tunning
randomcv_models = [
    ('KNN', KNeighborsRegressor(),knn_params),
    ('RF',RandomForestRegressor(),rf_params)
]

In [23]:
## Hyperparameter Tuning
from sklearn.model_selection import RandomizedSearchCV

model_param = {}

for name,model,params in randomcv_models:
    random=RandomizedSearchCV(
        estimator=model,
        param_distributions=params,
        n_iter=100,
        cv=3,
        verbose=2,
        n_jobs=1
    )

    random.fit(X_train,y_train)
    model_param[name]=random.best_params_

for model_name in model_param:
    print(f"-------------------Best Params for {model_name} ----------")
    print(model_param[model_name])

Fitting 3 folds for each of 6 candidates, totalling 18 fits


ValueError: Invalid parameter 'n_neighbours' for estimator KNeighborsRegressor(). Valid parameters are: ['algorithm', 'leaf_size', 'metric', 'metric_params', 'n_jobs', 'n_neighbors', 'p', 'weights'].